<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/llama_inference_zeroshot_chartqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install "torch==2.4.0" tensorboard pillow torchvision accelerate huggingface_hub
!pip -q install  --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen_vl_utils"

   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/797.2 MB 76.6 MB/s eta 0:00:10
ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "/usr/lib/python3.10/http/client.py", line 466, in read
    s = self.fp.read(amt)
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbyt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
import os
os.environ['HF_HOME'] = '/home/sa5u24/VQA'
hf_home = os.path.expanduser(
    os.getenv("HF_HOME", os.path.join(os.getenv("XDG_CACHE_HOME", "~/.cache"), "huggingface"))
)
print(hf_home)

from huggingface_hub import login

# Replace 'your-hf-token-here' with your actual Hugging Face token
login(token="")

In [ ]:
system_message = """You are a Vision Language Model specialized in interpreting visual data from chart images.
Your task is to analyze the provided chart image and respond to queries with concise answers, usually a single word, number, or short phrase.
The charts include a variety of types (e.g., line charts, bar charts) and contain colors, labels, and text.
Focus on delivering accurate, succinct answers based on the visual information. Avoid additional explanation unless absolutely necessary."""

def format_data(sample):
    return [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": system_message
                }
            ],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": sample["image"],
                },
                {
                    "type": "text",
                    "text": sample['query'],
                }
            ],
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": sample["label"][0]
                }
            ],
        },
    ]

from datasets import load_dataset

dataset_id = "HuggingFaceM4/ChartQA"
train_dataset, eval_dataset, test_dataset = load_dataset(dataset_id, split=['train[:10%]', 'val[:10%]', 'test[:10%]'])

train_dataset = [format_data(sample) for sample in train_dataset]
eval_dataset = [format_data(sample) for sample in eval_dataset]
test_dataset = [format_data(sample) for sample in test_dataset]

train_dataset[200], len(train_dataset), len(eval_dataset), len(test_dataset)

In [ ]:
import gc
import time

def clear_memory():
    # Delete variables if they exist in the current global scope
    if 'inputs' in globals(): del globals()['inputs']
    if 'model' in globals(): del globals()['model']
    if 'processor' in globals(): del globals()['processor']
    if 'trainer' in globals(): del globals()['trainer']
    if 'peft_model' in globals(): del globals()['peft_model']
    if 'bnb_config' in globals(): del globals()['bnb_config']
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")



def generate_text_from_sample(model, processor, sample, max_new_tokens=1024, device="cuda"):
    # Prepare the text input by applying the chat template
    text_input = processor.apply_chat_template(
        sample[1:2],  # Use the sample without the system message
        tokenize=False,
        add_generation_prompt=True
    )

    # Process the visual input from the sample
    # image_inputs, _ = process_vision_info(sample)
    image_inputs = sample[1]['content'][0]['image'].convert("RGB")
    # Prepare the inputs for the model
    model_inputs = processor(
        text=[text_input],
        images=image_inputs,
        return_tensors="pt",
    ).to(device)  # Move inputs to the specified device

    # Generate text with the model
    generated_ids = model.generate(**model_inputs, max_new_tokens=max_new_tokens)

    # Trim the generated ids to remove the input ids
    trimmed_generated_ids = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode the output text
    output_text = processor.batch_decode(
        trimmed_generated_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return output_text[0]  # Return the first decoded output text


In [ ]:
clear_memory()

GPU allocated memory: 0.01 GB
GPU reserved memory: 0.02 GB


In [ ]:
import evaluate
import torch
from nltk.translate.meteor_score import meteor_score, single_meteor_score
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import requests
from torch import nn
from transformers import MllamaForConditionalGeneration, AutoProcessor, MllamaConfig, AutoModelForCausalLM
from typing import List, Optional, Tuple, Union
from PIL import Image
import matplotlib.pyplot as plt
from qwen_vl_utils import process_vision_info
from transformers import BitsAndBytesConfig



quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # low_cpu_mem_usage=True,
    # bnb_4bit_compute_dtype=torch.float16
)

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
processor = AutoProcessor.from_pretrained(model_id)
model = MllamaForConditionalGeneration.from_pretrained(
            model_id, torch_dtype=torch.bfloat16, device_map="auto", low_cpu_mem_usage=True,
            quantization_config=quantization_config,
        )

all_pred = []
all_ans = []
model.eval()
with torch.no_grad():
    for sample in test_dataset:
        output = generate_text_from_sample(model, processor, sample)
        ans = sample[2]['content'][0]['text']
        all_pred.append(output)
        all_ans.append(ans)



In [ ]:
for i in range(len(all_pred)):
    print("pred:", all_pred[i])
    print("ans:", all_ans[i])

pred: There are 11 food items shown in the bar graph.
ans: 14
pred: To calculate the difference in value between Lamb and Corn, subtract the value of Corn from the value of Lamb.

• Lamb: 103.7
• Corn: 103.13

**Calculation:**
103.7 - 103.13 = 0.57

*Answer*: 0.57
ans: 0.57
pred: There are 3 bars shown in the chart.
ans: 3
pred: To determine if the value of Madagascar exceeds that of Fiji, a comparison of their respective values is necessary.

**Values:**

* Madagascar: 0.21%
* Fiji: 0.38%

**Result:**
Since 0.21% is less than 0.38%, the value of Madagascar is not greater than that of Fiji.

*Answer*: No
ans: No
pred: The lowest bar has a value of 23.
ans: 23
pred: To calculate the difference between the highest and lowest green bars, the following steps are taken:

* Identify the values of the green bars:
	+ Highest value: 29% in 2016
	+ Lowest value: 23% in 2015
* Calculate the difference by subtracting the lowest value from the highest value: 29 - 23 = 6

*Answer*: 6
ans: 6
pred: Ac

In [ ]:
rouge = evaluate.load("rouge")
rouge_results = rouge.compute(predictions= all_pred, references=all_ans)
print(rouge_results)


{'rouge1': 0.11616600174635018, 'rouge2': 0.02590092336534345, 'rougeL': 0.11651964551335531, 'rougeLsum': 0.11640484997088382}


In [ ]:
bleu_score = corpus_bleu(all_ans, all_pred, weights=(1.0, 0.0, 0.0, 0.0))
print(bleu_score)

0.0187011969745184


/home/sa5u24/anaconda3/envs/lit_llama/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/sa5u24/anaconda3/envs/lit_llama/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/sa5u24/anaconda3/envs/lit_llama/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contain

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

m_score=0
for line in zip(all_ans, all_pred):
    ref = word_tokenize(line[0])
    hypo = word_tokenize(line[1])
    m_score += meteor_score([ref], hypo)
meteors = m_score/len(all_ans)
print(meteors)

[nltk_data] Downloading package wordnet to /home/sa5u24/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sa5u24/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


0.1374583098003345
